# Efficient Yelp API Calls

- Darlene Adams

In [1]:
# Standard Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Additional Imports
import os, json, math, time
from yelpapi import YelpAPI
from tqdm.notebook import tqdm_notebook

In [2]:
# Load API Credentials
with open('/Users/dmusl/.secret/yelp_api.json') as f:
    login = json.load(f)

In [3]:
login.keys()

dict_keys(['client-id', 'api-key'])

In [4]:
# Instantiate YelpAPI Variable
yelp_api = YelpAPI(login['api-key'], timeout_s=5.0)

In [5]:
## Define Search Terms and File Paths

In [6]:
# set our API call parameters and filename before the first call
location = 'Cleveland, OH 44135'
term = 'crab'

In [7]:
location.split(',')[0]

'Cleveland'

In [8]:
# Specifying JSON_FILE filename (can include a folder)
# include the search terms in the filename
JSON_FILE = "Data/results_in_progress_cleveland_crab.json"
JSON_FILE

'Data/results_in_progress_cleveland_crab.json'

In [9]:
## Check if JSON_FILE exists
file_exists = os.path.isfile(JSON_FILE)
## If it does not exist: 
if file_exists == False:
    
    ## CREATE ANY NEEDED FOLDERS
    # Get the Folder Name only
    folder = os.path.dirname(JSON_FILE)
    ## If JSON_FILE included a folder:
    if len(folder)>0:
        # create the folder
        os.makedirs(folder,exist_ok=True)
        
        
    ## INFORM USER AND SAVE EMPTY LIST
    print(f'[i] {JSON_FILE} not found. Saving empty list to file.')
    
    
    # save an empty list
    with open(JSON_FILE,'w') as f:
        json.dump([],f)  
# If it exists, inform user
else:
    print(f"[i] {JSON_FILE} already exists.")

[i] Data/results_in_progress_cleveland_crab.json not found. Saving empty list to file.


In [10]:
## Load previous results and use len of results for offset
with open(JSON_FILE,'r') as f:
    previous_results = json.load(f)
    
## set offset based on previous results
n_results = len(previous_results)
print(f'- {n_results} previous results found.')

- 0 previous results found.


In [11]:
# use our yelp_api variable's search_query method to perform our API call
results = yelp_api.search_query(term = term, location = location, offset = n_results)

results.keys()

dict_keys(['businesses', 'total', 'region'])

In [12]:
## How many results total?
total_results = results['total']
total_results

382

In [13]:
## How many did we get the details for?
results_per_page = len(results['businesses'])
results_per_page

20

In [14]:
# Use math.ceil to round up for the total number of pages of results.
n_pages = math.ceil((results['total']-n_results)/ results_per_page)
n_pages

20

In [15]:
# join new results with old list with extend and save to file
previous_results.extend(results['businesses'])  
with open(JSON_FILE,'w') as f:
     json.dump(previous_results,f)

In [20]:
for i in tqdm_notebook( range(1,n_pages+1)):
    
    ## Read in results in progress file and check the length
    with open(JSON_FILE, 'r') as f:
        previous_results = json.load(f)
    ## save number of results for to use as offset
    n_results = len(previous_results)
    
    if (n_results + results_per_page) > 1000:
        print('Exceeded 1000 api calls. Stopping loop.')
        break
    
    ## use n_results as the OFFSET 
    results = yelp_api.search_query(location=location,
                                    term=term, 
                                    offset=n_results)
    
    
    
    ## append new results and save to file
    previous_results.extend(results['businesses'])
    
    # display(previous_results)
    with open(JSON_FILE,'w') as f:
        json.dump(previous_results,f)
        
    # add a 200ms pause
    time.sleep(.2)
    


  0%|          | 0/20 [00:00<?, ?it/s]

In [21]:
# load final results
final_df = pd.read_json(JSON_FILE)
display(final_df.head(), final_df.tail())

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,location,phone,display_phone,distance,price
0,Mry-o29iw1b8UGe5E5Tnfw,red-crab-juicy-seafood-cleveland,Red Crab Juicy Seafood,https://s3-media3.fl.yelpcdn.com/bphoto/PO5x-f...,False,https://www.yelp.com/biz/red-crab-juicy-seafoo...,47,"[{'alias': 'seafood', 'title': 'Seafood'}]",4.0,"{'latitude': 41.425456, 'longitude': -81.735635}",[pickup],"{'address1': '4754 Ridge Rd', 'address2': '', ...",+12164653888,(216) 465-3888,6749.291697,NaN
1,-d9JCrnjRW3Xmp2QvGwZ3g,seafood-shake-strongsville,Seafood Shake,https://s3-media1.fl.yelpcdn.com/bphoto/SNdn2d...,False,https://www.yelp.com/biz/seafood-shake-strongs...,16,"[{'alias': 'seafood', 'title': 'Seafood'}]",4.5,"{'latitude': 41.313691, 'longitude': -81.814785}","[pickup, delivery]","{'address1': '16532 Royalton Rd', 'address2': ...",,,12173.122768,NaN
2,EYqNh5zMIE2EDQ43ciazcA,lees-seafood-boil-north-olmsted-north-olmsted,Lee's Seafood Boil North Olmsted,https://s3-media4.fl.yelpcdn.com/bphoto/8X1m9B...,False,https://www.yelp.com/biz/lees-seafood-boil-nor...,35,"[{'alias': 'seafood', 'title': 'Seafood'}, {'a...",4.0,"{'latitude': 41.43291, 'longitude': -81.88364}","[pickup, delivery]","{'address1': '23642 Lorain Rd', 'address2': No...",+14404559003,(440) 455-9003,5702.722456,NaN
3,CymM3BP68hGigBK_5ap58A,sarita-a-restaurant-lakewood-2,sarita a restaurant,https://s3-media2.fl.yelpcdn.com/bphoto/ITKzFX...,False,https://www.yelp.com/biz/sarita-a-restaurant-l...,258,"[{'alias': 'newamerican', 'title': 'New Americ...",4.5,"{'latitude': 41.47694, 'longitude': -81.796886}",[delivery],"{'address1': '14523 Madison Ave', 'address2': ...",+12162265200,(216) 226-5200,6205.255053,$$
4,0I0yATfnNQJB-A-IPGgqDA,aristo-bistro-westlake-3,Aristo Bistro,https://s3-media2.fl.yelpcdn.com/bphoto/muWlSo...,False,https://www.yelp.com/biz/aristo-bistro-westlak...,113,"[{'alias': 'chinese', 'title': 'Chinese'}, {'a...",4.5,"{'latitude': 41.4571533123911, 'longitude': -8...",[delivery],"{'address1': '25124 Center Ridge Rd', 'address...",+14404559900,(440) 455-9900,7803.604542,$$


,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,location,phone,display_phone,distance,price
369,123cbSJgE-BzNCzXV_7r0g,north-coast-cafe-cleveland,North Coast Cafe,https://s3-media4.fl.yelpcdn.com/bphoto/U2Yhaj...,False,https://www.yelp.com/biz/north-coast-cafe-clev...,23,"[{'alias': 'newamerican', 'title': 'New Americ...",3.0,"{'latitude': 41.5019874572754, 'longitude': -8...",[delivery],"{'address1': '9801 Carnegie Ave', 'address2': ...",+12167074051,(216) 707-4051,18648.359224,$$
370,r0DureDzsHpzs_VZem5k7g,outback-steakhouse-westlake,Outback Steakhouse,https://s3-media4.fl.yelpcdn.com/bphoto/MMfyJt...,False,https://www.yelp.com/biz/outback-steakhouse-we...,133,"[{'alias': 'steak', 'title': 'Steakhouses'}]",2.5,"{'latitude': 41.4749211698861, 'longitude': -8...","[pickup, delivery]","{'address1': '24900 Sperry Dr', 'address2': ''...",+14408923445,(440) 892-3445,8752.396747,$$
371,XUA7xA7myMaCuN6G3xYdIA,number-one-pho-cleveland,Number One Pho,https://s3-media4.fl.yelpcdn.com/bphoto/-OpBf0...,False,https://www.yelp.com/biz/number-one-pho-clevel...,270,"[{'alias': 'vietnamese', 'title': 'Vietnamese'}]",3.5,"{'latitude': 41.5107498168945, 'longitude': -8...","[pickup, delivery]","{'address1': '3120 Superior Ave E', 'address2'...",+12167811176,(216) 781-1176,15683.243615,$$
372,4rSHsXO4ozQoD0hoPCDatQ,china-express-lakewood,China Express,https://s3-media4.fl.yelpcdn.com/bphoto/TN4yqQ...,False,https://www.yelp.com/biz/china-express-lakewoo...,41,"[{'alias': 'chinese', 'title': 'Chinese'}]",2.5,"{'latitude': 41.4851178, 'longitude': -81.7868...","[pickup, delivery]","{'address1': '13621 Detroit Ave', 'address2': ...",+12162277890,(216) 227-7890,7313.007286,$
373,6tY0tn39Mb8FCLYBAXXOUw,china-renaissance-north-olmsted,China Renaissance,https://s3-media3.fl.yelpcdn.com/bphoto/rcqRLJ...,False,https://www.yelp.com/biz/china-renaissance-nor...,63,"[{'alias': 'chinese', 'title': 'Chinese'}]",3.5,"{'latitude': 41.407014, 'longitude': -81.946028}","[pickup, delivery]","{'address1': '29336 Lorain Rd', 'address2': ''...",+14407345888,(440) 734-5888,10943.813065,$$


In [22]:
# check for duplicate ID's 
final_df.duplicated(subset='id').sum()

2

In [23]:
## Drop duplicate ids and confirm there are no more duplicates
final_df = final_df.drop_duplicates(subset='id')
final_df.duplicated(subset='id').sum()

0

In [24]:
# save the final results to a compressed csv
final_df.to_csv('Data/results_in_progress_cleveland_crab.csv', compression='gzip',index=False)